In [ ]:
pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 7.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import Sastrawi
import numpy as np
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
from google.colab import files
files.upload()

Saving datates_simpatika.csv to datates_simpatika.csv


{'datates_simpatika.csv': b'content,score,aspek,sentimen\r\naplikasi ini sangat membantu dalam pekerjaan,5,Kepraktisan,Positif  \r\nfiturnya lengkap dan mudah digunakan,5,Fitur,Positif  \r\nsering error saat login,2,Autentikasi,Negatif  \r\ntampilannya menarik dan user-friendly,5,Performa,Positif  \r\nbutuh akses internet yang stabil,3,Kepraktisan,Netral  \r\nmembantu sekali dalam mengelola data,5,Kepraktisan,Positif  \r\nloading lama saat membuka aplikasi,2,Performa,Negatif  \r\nbagus dan sangat bermanfaat,5,Umum,Positif  \r\nbanyak iklan yang mengganggu,2,Iklan,Negatif  \r\nterkadang sulit untuk login,3,Autentikasi,Netral  \r\nmempermudah pekerjaan guru,5,Kepraktisan,Positif  \r\naplikasi ini sering crash,2,Performa,Negatif  \r\ninformasi yang diberikan sangat akurat,5,Informasi,Positif  \r\nbutuh pembaruan agar lebih stabil,3,Performa,Netral  \r\nmudah digunakan oleh semua orang,5,Kepraktisan,Positif  \r\niklan terlalu sering muncul,2,Iklan,Negatif  \r\nsangat membantu untuk mencari

In [ ]:
df = pd.read_csv("sentiment (1).csv")
df

,content,score,aspek,sentimen
0,sangat bagus untuk membantu bapak ibu guru dal...,5,Informasi,Positif
1,sangat membantu app ini karena kami membutuhka...,5,Informasi,Positif
2,aplikasi paling top keren banget lengkap yang ...,5,Umum,Positif
3,alhamdulillah sangat membantu,5,Umum,Positif
4,saya berikan bintang karena semua ikhtiar demi...,5,Umum,Positif
...,...,...,...,...
1636,bagus,1,Umum,Netral
1637,ok,1,Umum,Netral
1638,sukses,1,Umum,Netral
1639,sip,1,Umum,Netral


### Stopword

In [ ]:
more_stop_words = []

stop_words = StopWordRemoverFactory().get_stop_words()

if "ok" in stop_words:
    stop_words.remove("ok")

new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text


df['content'] = df['content'].apply(lambda x: stopword(x))

### Tokenize

In [ ]:
tokenized = df['content'].apply(lambda x:x.split())
tokenized

,content
0,"[sangat, bagus, membantu, bapak, ibu, guru, me..."
1,"[sangat, membantu, app, karena, membutuhkan, i..."
2,"[aplikasi, paling, top, keren, banget, lengkap..."
3,"[alhamdulillah, sangat, membantu]"
4,"[berikan, bintang, semua, ikhtiar, kebaikan, w..."
...,...
1636,[bagus]
1637,[ok]
1638,[sukses]
1639,[sip]


### Stemming

In [ ]:
def stemming(text_cleaning):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in text_cleaning:
    if w.lower() == "semoga":
      dt = w
    else:
      dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = " ".join(do)
  print(d_clean)
  return d_clean

tokenized = tokenized.apply(stemming)

sangat bagus bantu bapak ibu guru lihat data cepat
sangat bantu app karena butuh informasi trimakasih simpatika
aplikasi paling top keren banget lengkap buat aplikasi pintar banget beri cerdas ilmu mau bagi terima kasih semoga sukses selalu
alhamdulillah sangat bantu
ikan bintang semua ikhtiar baik walaupun perlu kembang ikan apresiasi setinggi-tingginya terima kasih app sangat manfaat kami guru seluruh nusantara
kasih full bintang ada syarat hehe mudah ketika lupa password sudah bagai cara apa saya ikut instruksi reset password belum hasil hingga sekarang
aplikasi simpel mudah paham mudah masa depan aplikasi simpatika beri banyak solusi kbm lingkung kemenag
mudah kerja utama apabila salah isi data mudah baik
layan sangat bagus bantu siap dokumen
terima kasih ada app bisa bantu cepat pantau data dan dapat informasi kemenag
siap simpatika layan komunikasi sangat manfaat guru kait informasi kemenag mudah guru bawah naung kemenag selalu dapat informasi aktual dapat dipertanggungjawabkan s

In [ ]:
tokenized = tokenized.to_frame()

In [ ]:
at1 = tokenized["content"].reset_index(drop=True)
at2 = df["aspek"].reset_index(drop=True)
at3 = df["sentimen"].reset_index(drop=True)
result = pd.concat([at1, at2, at3], axis=1)

In [ ]:
from collections import Counter

# Cek distribusi awal
print("Distribusi awal:", Counter(result['sentimen']))

Distribusi awal: Counter({'Positif': 1287, 'Negatif': 230, 'Netral': 124})


In [ ]:
# Imbalance Handling aspek

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Undersampling kelas yang memiliki lebih dari 400 baris
undersampler = RandomUnderSampler(sampling_strategy={k: 400 for k, v in Counter(result['aspek']).items() if v > 400}, random_state=42)
X_under, y_under = undersampler.fit_resample(result[['content']], result['aspek'])

# Oversampling kelas yang memiliki kurang dari 400 baris
oversampler = RandomOverSampler(sampling_strategy={k: 400 for k, v in Counter(y_under).items() if v < 400}, random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_under, y_under)

In [ ]:
# Imbalance Handling sentimen

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Undersampling kelas yang memiliki lebih dari 700 baris
undersampler = RandomUnderSampler(sampling_strategy={k: 700 for k, v in Counter(result['sentimen']).items() if v > 700}, random_state=42)
X_under, y_under = undersampler.fit_resample(result[['content']], result['sentimen'])

# Oversampling kelas yang memiliki kurang dari 700 baris
oversampler = RandomOverSampler(sampling_strategy={k: 700 for k, v in Counter(y_under).items() if v < 700}, random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_under, y_under)

In [ ]:
# Cek distribusi akhir
print("Distribusi setelah resampling:", Counter(y_resampled))

Distribusi setelah resampling: Counter({'Negatif': 700, 'Netral': 700, 'Positif': 700})


In [ ]:
X_resampled = X_resampled['content']

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [ ]:
y_encoded = label_encoder.fit_transform(y_resampled)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_encoded,
                                                    test_size=0.3, stratify=y_resampled, random_state=30)

In [ ]:
X_train = X_resampled
y_train = y_encoded

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(decode_error='replace', encoding='utf-8')

In [ ]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
X_train

<2100x707 sparse matrix of type '<class 'numpy.float64'>'
	with 8163 stored elements in Compressed Sparse Row format>

In [ ]:
X_train = X_train.toarray()
X_test = X_test.toarray()

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

cv_method = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=999)

params_NB = {'var_smoothing': np.logspace(0, -9, num=100)}
gscv_nb = GridSearchCV(estimator=nb,
                        param_grid=params_NB,
                        cv = cv_method,
                        verbose = 1,
                        scoring = 'accuracy')

gscv_nb.fit(X_train, y_train)
gscv_nb.best_params_

Fitting 15 folds for each of 100 candidates, totalling 1500 fits


{'var_smoothing': 0.03511191734215131}

In [ ]:
nb = GaussianNB(var_smoothing=  0.03511191734215131)

In [ ]:
nb.fit(X_train, y_train)

GaussianNB(var_smoothing=0.03511191734215131)

In [ ]:
y_pred_nb = nb.predict(X_test)

In [ ]:
y_pred_nb

array([3, 0, 6, 3, 6, 2, 4, 6, 5, 6, 6, 0, 2, 2, 1, 0, 1, 1, 0, 2, 6, 0,
       3, 1, 1, 3, 0, 0, 2, 4, 4, 6, 6, 6, 2, 0, 0, 5, 0, 6, 3, 5, 0, 4,
       5, 6, 1, 2, 1, 3, 4, 6, 6, 4, 2, 2, 2, 5, 5, 0, 0, 6, 6, 0, 5, 2,
       4, 6, 0, 1, 0, 6, 4, 4, 3, 6, 5, 0, 2, 6, 5, 2, 2, 6, 3, 3, 5, 4,
       1, 2, 5, 5, 5, 0, 6, 4, 5, 2, 5, 5, 5, 5, 4, 0, 3, 1, 2, 5, 5, 3,
       1, 2, 6, 3, 6, 5, 5, 3, 2, 6, 1, 3, 1, 2, 5, 2, 6, 5, 5, 5, 0, 5,
       6, 0, 3, 2, 0, 5, 6, 5, 6, 6, 1, 5, 2, 2, 4, 5, 5, 5, 6, 2, 2, 5,
       1, 5, 3, 0, 6, 2, 6, 2, 4, 3, 5, 4, 3, 1, 1, 0, 3, 2, 2, 1, 0, 0,
       6, 6, 2, 6, 3, 1, 0, 6, 0, 5, 5, 3, 6, 4, 1, 1, 4, 3, 5, 5, 1, 6,
       6, 0, 6, 5, 2, 0, 3, 5, 6, 1, 5, 1, 6, 2, 2, 2, 5, 5, 3, 2, 5, 0,
       3, 6, 3, 0, 0, 2, 2, 5, 3, 0, 5, 2, 1, 3, 6, 3, 2, 1, 1, 4, 2, 5,
       3, 2, 1, 4, 2, 1, 4, 3, 4, 1, 1, 0, 3, 3, 3, 6, 2, 3, 1, 1, 6, 1,
       0, 6, 6, 1, 4, 0, 3, 5, 6, 0, 2, 1, 2, 5, 4, 3, 6, 1, 4, 4, 4, 6,
       2, 0, 0, 1, 3, 2, 6, 0, 3, 5, 0, 0, 1, 5, 4,

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_nb)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8786


### SVM

In [ ]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train, y_train)

y_pred_svm = svm.predict(X_test)
y_pred_svm

array([2, 1, 2, 0, 2, 0, 0, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 0, 0, 1, 1,
       0, 0, 0, 2, 1, 2, 1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1, 0, 0, 0, 0,
       2, 1, 2, 1, 2, 0, 0, 2, 0, 0, 1, 0, 1, 1, 0, 0, 0, 2, 0, 0, 2, 1,
       1, 0, 0, 0, 0, 2, 0, 2, 1, 1, 1, 0, 1, 2, 1, 0, 0, 2, 1, 2, 1, 1,
       2, 2, 1, 2, 1, 1, 2, 1, 0, 0, 0, 1, 1, 2, 2, 2, 0, 2, 1, 0, 0, 1,
       1, 1, 2, 2, 0, 0, 2, 1, 2, 0, 0, 1, 1, 2, 2, 0, 0, 2, 2, 1, 0, 0,
       1, 0, 0, 0, 1, 2, 2, 2, 2, 1, 2, 1, 0, 1, 1, 2, 2, 2, 1, 2, 1, 2,
       1, 2, 2, 1, 0, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 0, 0, 2, 0, 2, 2,
       0, 0, 1, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1, 2, 2, 2, 1, 1, 0, 2, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 2, 0, 1, 2, 0, 1, 1, 1, 2, 0, 0, 0,
       0, 0, 0, 1, 2, 2, 2, 1, 1, 0, 0, 2, 0, 0, 0, 2, 2, 0, 1, 1, 0, 1,
       2, 2, 0, 2, 2, 1, 0, 2, 1, 0, 2, 1, 0, 0, 1, 0, 2, 1, 1, 1, 2, 2,
       2, 1, 0, 0, 1, 2, 1, 1, 0, 2, 2, 2, 2, 1, 2, 0, 2, 1, 2, 2, 0, 1,
       1, 1, 0, 2, 1, 0, 1, 0, 1, 0, 1, 2, 2, 1, 1,

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_svm)
print(f"Accuracy: {accuracy:.4f}")

# Akurasi aspek = 0,98
# Akurasi sentimen = 0,94

Accuracy: 0.9413


In [ ]:
import pickle

# Simpan model ke file
with open('svm_model_sentimen.pkl', 'wb') as f:
    pickle.dump(svm, f)

### Prediksi

In [ ]:
df_tes = pd.read_csv("datates_simpatika.csv")
df_tes

,content,score,aspek,sentimen
0,aplikasi ini sangat membantu dalam pekerjaan,5,Kepraktisan,Positif
1,fiturnya lengkap dan mudah digunakan,5,Fitur,Positif
2,sering error saat login,2,Autentikasi,Negatif
3,tampilannya menarik dan user-friendly,5,Performa,Positif
4,butuh akses internet yang stabil,3,Kepraktisan,Netral
...,...,...,...,...
96,aplikasi ini ringan dan cepat,5,Performa,Positif
97,banyak fitur yang tidak bisa digunakan,2,Fitur,Negatif
98,iklan terlalu berlebihan,2,Iklan,Negatif
99,butuh lebih banyak opsi integrasi,4,Fitur,Netral


In [ ]:
more_stop_words = []

stop_words = StopWordRemoverFactory().get_stop_words()

if "ok" in stop_words:
    stop_words.remove("ok")

new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text


df_tes['content'] = df_tes['content'].apply(lambda x: stopword(x))

### Tokenize

In [ ]:
tokenized = df_tes['content'].apply(lambda x:x.split())
tokenized

,content
0,"[aplikasi, sangat, membantu, pekerjaan]"
1,"[fiturnya, lengkap, mudah, digunakan]"
2,"[sering, error, login]"
3,"[tampilannya, menarik, user-friendly]"
4,"[butuh, akses, internet, stabil]"
...,...
96,"[aplikasi, ringan, cepat]"
97,"[banyak, fitur, tidak, digunakan]"
98,"[iklan, terlalu, berlebihan]"
99,"[butuh, lebih, banyak, opsi, integrasi]"


### Stemming

In [ ]:
def stemming(text_cleaning):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in text_cleaning:
    if w.lower() == "semoga":
      dt = w
    else:
      dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = " ".join(do)
  print(d_clean)
  return d_clean

tokenized = tokenized.apply(stemming)

aplikasi sangat bantu kerja
fiturnya lengkap mudah guna
sering error login
tampil tarik user-friendly
butuh akses internet stabil
bantu sekali kelola data
loading lama buka aplikasi
bagus sangat manfaat
banyak iklan ganggu
terkadang sulit login
mudah kerja guru
aplikasi sering crash
informasi beri sangat akurat
butuh baru lebih stabil
mudah guna semua orang
iklan terlalu sering muncul
sangat bantu cari data
fitur-fiturnya canggih update
login sering gagal
terkadang lambat muat halaman
desain simpel enak pakai
data tampil kurang lengkap
aplikasi sangat lambat
sangat praktis guna
suka tiba keluar sendiri
informasi saji kurang update
butuh tingkat cepat
fitur cari sangat bantu
aplikasi terlalu berat hp lama
iklan muncul sedang kerja
mudah akses data penting
terkadang susah akses
user interface sangat nyaman
banyak bug perlu baik
bisa login update
cukup baik masih bug
butuh fitur tambah lapor
respon aplikasi sangat cepat
bantu kerja administrasi
tampil modern responsif
iklan terlalu ganggu

In [ ]:
tokenized = tokenized.to_frame()

In [ ]:
X_new = tokenized

In [ ]:
X_new = vectorizer.transform(X_new)

In [ ]:
X_new = X_new.toarray()

In [ ]:
X_new

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
y_pred2 = svm.predict(X_new)
y_pred2

array([2, 0, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 2, 0,
       2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 1, 0, 1, 2, 2, 2, 0, 0, 0, 2, 0,
       0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 2,
       2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0,
       2, 2, 2, 2, 0, 2, 1, 2, 2, 0, 0, 2, 2])

In [ ]:
y_decoded = label_encoder.inverse_transform(y_pred2)
y_decoded

array(['Positif', 'Negatif', 'Negatif', 'Positif', 'Positif', 'Positif',
       'Negatif', 'Positif', 'Negatif', 'Negatif', 'Positif', 'Negatif',
       'Positif', 'Positif', 'Positif', 'Negatif', 'Positif', 'Negatif',
       'Negatif', 'Negatif', 'Positif', 'Negatif', 'Positif', 'Positif',
       'Negatif', 'Negatif', 'Positif', 'Positif', 'Negatif', 'Negatif',
       'Positif', 'Positif', 'Positif', 'Netral', 'Negatif', 'Netral',
       'Positif', 'Positif', 'Positif', 'Negatif', 'Negatif', 'Negatif',
       'Positif', 'Negatif', 'Negatif', 'Positif', 'Negatif', 'Positif',
       'Positif', 'Negatif', 'Positif', 'Positif', 'Positif', 'Negatif',
       'Positif', 'Negatif', 'Negatif', 'Positif', 'Negatif', 'Positif',
       'Negatif', 'Positif', 'Negatif', 'Positif', 'Positif', 'Positif',
       'Positif', 'Positif', 'Negatif', 'Positif', 'Negatif', 'Positif',
       'Positif', 'Negatif', 'Positif', 'Positif', 'Positif', 'Positif',
       'Positif', 'Positif', 'Positif', 'Positif', 'P

In [ ]:
y_new = df_tes['sentimen'].str.strip()
y_neww = y_new.to_numpy()

In [ ]:
y_neww
# y_decoded.shape

array(['Positif', 'Positif', 'Negatif', 'Positif', 'Netral', 'Positif',
       'Negatif', 'Positif', 'Negatif', 'Netral', 'Positif', 'Negatif',
       'Positif', 'Netral', 'Positif', 'Negatif', 'Positif', 'Positif',
       'Negatif', 'Netral', 'Positif', 'Netral', 'Negatif', 'Positif',
       'Negatif', 'Netral', 'Netral', 'Positif', 'Negatif', 'Negatif',
       'Positif', 'Netral', 'Positif', 'Negatif', 'Negatif', 'Netral',
       'Netral', 'Positif', 'Positif', 'Positif', 'Netral', 'Positif',
       'Positif', 'Negatif', 'Netral', 'Positif', 'Netral', 'Positif',
       'Positif', 'Negatif', 'Netral', 'Positif', 'Positif', 'Negatif',
       'Positif', 'Netral', 'Netral', 'Positif', 'Positif', 'Positif',
       'Netral', 'Netral', 'Negatif', 'Positif', 'Positif', 'Netral',
       'Positif', 'Positif', 'Negatif', 'Positif', 'Netral', 'Positif',
       'Positif', 'Netral', 'Positif', 'Netral', 'Positif', 'Netral',
       'Positif', 'Netral', 'Negatif', 'Positif', 'Netral', 'Positif',
   

In [ ]:
accuracy = accuracy_score(y_neww, y_decoded)
print(f"Accuracy: {accuracy:.4f}")

# Aspek = 0,55
# Senitmen = 0,62


Accuracy: 0.6238


In [ ]:
y_decoded_df = pd.DataFrame(y_decoded, columns=['Label'])
y_decoded_df

,Label
0,Positif
1,Negatif
2,Negatif
3,Positif
4,Positif
...,...
96,Positif
97,Negatif
98,Negatif
99,Positif


In [ ]:
df_result = pd.concat([df_tes['content'], y_decoded_df], axis=1)
df_result

,content,Label
0,aplikasi sangat membantu pekerjaan,Positif
1,fiturnya lengkap mudah digunakan,Negatif
2,sering error login,Negatif
3,tampilannya menarik user-friendly,Positif
4,butuh akses internet stabil,Positif
...,...,...
96,aplikasi ringan cepat,Positif
97,banyak fitur tidak digunakan,Negatif
98,iklan terlalu berlebihan,Negatif
99,butuh lebih banyak opsi integrasi,Positif
